In [39]:
from selenium import webdriver
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By
from parsel import Selector
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import requests
from bs4 import BeautifulSoup
import re
from statistics import mean

driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())

[WDM] - Downloading: 16.2kB [00:00, 1.85MB/s]                   
C:\Users\user\AppData\Local\Temp\ipykernel_15000\969000144.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())


In [40]:
driver.get("https://www.prisjakt.nu/produkt.php?p=5688342")
try:
    element1 = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.BannerCookieBanner div.BannerContentCookieBanner div.ButtonsWrapperCookieBanner.AcceptButtonWrapperCookieBanner button.AcceptButtonCookieBanner.ButtonCookieBanner span.TextWrapperCookieBanner > span:nth-child(1)"))
        )
    element1.click()
except:
    element1 = None
try:
    element2 = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.App-sc-2fu3f8-0.ljZBoE section.Main-sc-2fu3f8-1.hnCjSa:nth-child(5) div.Content-sc-2fu3f8-2.hybPGh div.PageContent-sc-1wgu331-5.dfQdce div.Wrapper-sc-1wgu331-1.gPsMfp div.Content-sc-1wgu331-0.gMPJqu div.ProductPage-sc-mnx6sq-0.iMqgaO div.SectionWrapper-sc-ia0zhw-0.bqrwQK:nth-child(1) div.PageSectionWrapper-sc-o2pbvx-0.gHOcYt.StyledPageSection-sc-1w3nqup-0.UUqee:nth-child(1) section.PageSectionContent-sc-o2pbvx-1.euuGdp section:nth-child(1) div.PriceListPanelWrapper-sc-1htx0wh-0.lkPiWw div.PricePanel-sc-wkzg9v-1.kSsiTe div:nth-child(3) > button.BaseButton--uexw3i.cVdzxy.textbutton"))
        )
    element2.click()
except:
    element2 = None

In [52]:
def get_prices():
    page_source =driver.page_source
    soup = BeautifulSoup(page_source,'html.parser')
    offers  = soup.find("ul", {"class":"PriceList-sc-wkzg9v-0 fbrkVc"})
    price_list = []
    for offer in offers:
        offer_prices  = offer.find_all("div", {"class":"PriceWrapper-sc-k40pbc-4 OYetQ"})
        price_list.append(offer_prices)

    price_list = [price for sublist in price_list for price in sublist]
    for i in range(0, len(price_list)):
        price_list[i] = price_list[i].text.replace('\n',"")

    for i in range(0, len(price_list)):
        price_list[i] = re.sub('[\xa0]', '',price_list[i])
        price_list[i] = re.sub('[kr]', '', price_list[i])
        price_list[i]= price_list[i].replace(r'[()]',"")

    for i in range(0, len(price_list)):
        price_list[i] = int(price_list[i])
    no_offers = len(price_list)
    avg_price =  mean(price_list)
    return no_offers, avg_price

In [57]:
avg_price = get_prices()[1]
avg_price

28790.75

In [ ]:
for i in range(0, len(price_list)):
    price_list[i] = re.sub('[\xa0]', '',price_list[i])
    price_list[i] = re.sub('[kr]', '', price_list[i])
    price_list[i]= price_list[i].replace(r'[()]',"")


In [1]:
class PropertiesPrices(object):
    def __init__(self):
        self.driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())

    def get_properties(self):
        url = link+"#properties"
        self.driver.get(url)
        try:
            element1 = WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//body/div[@id='root']/div[@class='App-sc-2fu3f8-0 ljZBoE']/section[@class='Main-sc-2fu3f8-1 hnCjSa']/div[@class='Content-sc-2fu3f8-2 hybPGh']/div[@class='PageContent-sc-1wgu331-5 dfQdce']/div[@class='Wrapper-sc-1wgu331-1 gPsMfp']/div[@class='Content-sc-1wgu331-0 gMPJqu']/div[@class='ProductPage-sc-mnx6sq-0 iMqgaO']/div/div[@id='#properties']/div[@class='PageSectionWrapper-sc-o2pbvx-0 kQdrfN']/section[@class='PageSectionContent-sc-o2pbvx-1 hbBYkf']/section[@data-test='PropertiesTabContent']/div[@class='StyledList--ji0zot McirA StyledPanel--17i5iqi ffQfPu']/div[@data-test-type='product-info']/div[@class='StyledViewport-sc-7zjdbj-0 iDfoDl']/section[1]/div[1]/div[1]"))
            )
            element1.click()
        except:
            element1 = None
        self.driver.find_element(By.XPATH, "//a[@class='TabLink-sc-4sd16e-3 jJHqPc']")
        page_source =self.driver.page_source
        soup = BeautifulSoup(page_source,'html.parser')
        specs = soup.find_all("section", {"class":"Section-sc-1jnk5ag-1 kLbzuK"})
        prop_names = []
        prop_vals = []
        for sect in specs:
            sections = sect.find_all("div", {"class", "Row-sc-1jnk5ag-3 jQedgK"})
            for section in sections:
                properties = section.find_all("div", {"class":"Column-sc-1jnk5ag-2 bIOAXd"})
                for property in properties:
                    property_name = property.find("span", {"class":"Text--bzqghn wmwsE bodysmalltext PropertyName-sc-1jnk5ag-5 hEdIXF"})
                    property_value = property.find("span", {"class":"Text--bzqghn ddqBq bodysmalltext PropertyValue-sc-1jnk5ag-6 etcKHO"})
                    if property_name !=None:
                        prop_names.append(property_name.text.replace('\n',""))
                    elif property_value !=None:
                        prop_vals.append(property_value.text.replace('\n',""))
        properties = {}
        for name in prop_names:
            for val in prop_vals:
                properties[name] = val
                prop_vals.remove(val)
                break

        return properties

    def get_prices(self):
        self.driver.get(link)
        page_source = self.driver.page_source
        soup = BeautifulSoup(page_source,'html.parser')
        offers  = soup.find("ul", {"class":"PriceList-sc-wkzg9v-0 fbrkVc"})
        price_list = []
        for offer in offers:
            offer_prices  = offer.find_all("div", {"class":"PriceWrapper-sc-k40pbc-4 OYetQ"})
            price_list.append(offer_prices)

        price_list = [price for sublist in price_list for price in sublist]
        for i in range(0, len(price_list)):
            price_list[i] = price_list[i].text.replace('\n',"")

        for i in range(0, len(price_list)):
            price_list[i] = re.sub('[\xa0]', '',price_list[i])
            price_list[i] = re.sub('[kr]', '', price_list[i])
            price_list[i]= price_list[i].replace(r'[()]',"")

        for i in range(0, len(price_list)):
            price_list[i] = int(price_list[i])
        no_offers = len(price_list)
        avg_price =  mean(price_list)

        return no_offers, avg_price

In [ ]:
prices = PropertiesPrices()
